In [9]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

In [10]:
#SE IMPORTA EL CSV
toyota_data = pd.read_csv('../CSV/Toyota2.csv')
toyota_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,118.000000,119.330002,116.500000,116.709999,116.709999,398100
1,2020-04-02,115.910004,118.120003,115.720001,117.820000,117.820000,292600
2,2020-04-03,114.809998,115.690002,113.980003,115.000000,115.000000,337100
3,2020-04-06,120.000000,122.809998,119.400002,122.410004,122.410004,328200
4,2020-04-07,125.570000,125.669998,121.900002,121.900002,121.900002,239600
...,...,...,...,...,...,...,...
980,2024-02-23,235.000000,235.759995,234.339996,235.000000,235.000000,279000
981,2024-02-26,237.589996,238.779999,236.539993,238.130005,238.130005,332900
982,2024-02-27,238.190002,238.839996,236.589996,238.039993,238.039993,301000
983,2024-02-28,237.929993,238.190002,237.000000,237.289993,237.289993,168700


In [11]:
#LIMPIEZA DATOS
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
toyota_data = toyota_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
toyota_data

,Date,Adj Close
0,2020-04-01,116.709999
1,2020-04-02,117.820000
2,2020-04-03,115.000000
3,2020-04-06,122.410004
4,2020-04-07,121.900002
...,...,...
980,2024-02-23,235.000000
981,2024-02-26,238.130005
982,2024-02-27,238.039993
983,2024-02-28,237.289993


In [12]:
numero_periodos = 5
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(toyota_data)):
    percent = (toyota_data['Adj Close'][i] - toyota_data['Adj Close'][i-1])/toyota_data['Adj Close'][i-1]
    percentage.append(percent)
toyota_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
toyota_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
toyota_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(toyota_data)-1):
    next_state_column.append(toyota_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(toyota_data)-1):
    previows_state_column.append(toyota_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
toyota_data = toyota_data.drop(toyota_data.index[-1])
toyota_data.insert(5, 'Next State', next_state_column)
toyota_data.insert(4, 'Previous State', previows_state_column)

# complete_table = AsciiTable([ford_data.columns]+list(ford_data.values))
# print("Complete Table \n",complete_table.table)

ford_trim=toyota_data.iloc[21:]
table_week_21 = AsciiTable([ford_trim.columns]+list(ford_trim.values))



ford_trim = ford_trim.copy()

# Calculate the number of rows per period
rows_per_period = len(ford_trim) // numero_periodos

# Create a new column to store the period
ford_trim['Period'] = 0

# Assign the period to each row based on the index
for i in range(5):
    start_index = i * rows_per_period
    end_index = (i + 1) * rows_per_period
    ford_trim.loc[start_index:end_index, 'Period'] = i + 1

# Correct the last period to be 5 instead of 0
ford_trim.loc[end_index:, 'Period'] = 5

# Access the AsciiTable object using an array divided by the period
table_by_period = []
for period in range(1, numero_periodos + 1):
    period_data = ford_trim[ford_trim['Period'] == period]
    table = AsciiTable([period_data.columns] + period_data.values.tolist())
    table_by_period.append(table)


for period, table in enumerate(table_by_period, start=1):
    print(f"Table for Period {period}\n{table.table}")





Table for Period 1
+------------+------------+-------------------------+-----------------------+----------------+---------------+------------+--------+
| Date       | Adj Close  | Change %                | std dev               | Previous State | Current State | Next State | Period |
+------------+------------+-------------------------+-----------------------+----------------+---------------+------------+--------+
| 2020-05-01 | 121.120003 | -0.019985379722805414   | 0.021916077847500135  | B2             | B1            | B1         | 1      |
| 2020-05-04 | 120.699997 | -0.003467684854664351   | 0.021163908829415348  | B1             | B1            | S1         | 1      |
| 2020-05-05 | 121.959999 | 0.010439122049025404    | 0.015897996955057607  | B1             | S1            | B1         | 1      |
| 2020-05-06 | 121.580002 | -0.003115751091470598   | 0.01588606480984306   | S1             | B1            | B1         | 1      |
| 2020-05-07 | 121.519997 | -0.0004935433378261472

In [13]:
#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):
    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if index > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states

In [14]:
frecuencias_observadas = []

table_by_period = np.array_split(ford_trim, numero_periodos)

for i in range(numero_periodos):
    # Calcula la tabla de frecuencias observadas usando crosstab entre el estado actual y el estado anterior (shift)
    frecuencia_obs = pd.crosstab(table_by_period[i]['Current State'].shift(), table_by_period[i]['Current State'], normalize=False)
    print("FRECUENCIA OBSERVADA - PERIODO", i+1, "\n", frecuencia_obs, "\n")
    # Agrega la frecuencia observada a la lista
    frecuencias_observadas.append(frecuencia_obs)


FRECUENCIA OBSERVADA - PERIODO 1 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             26   0   4  28   7   1
B2              6   5   1   2   1   2
B3              2   0   0   3   3   0
S1             26   7   2  25   8   0
S2              5   3   1   7   6   4
S3              1   2   0   3   1   0 

FRECUENCIA OBSERVADA - PERIODO 2 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             11   8   0  31   5   0
B2              5   1   1   9   5   1
B3              1   2   0   2   0   0
S1             27   7   4  23  12   3
S2              9   3   1  10   4   1
S3              2   1   0   1   1   1 

FRECUENCIA OBSERVADA - PERIODO 3 
 Current State  B1  B2  B3  S1  S2  S3
Current State                        
B1             25   9   3  22   8   0
B2              7   1   2  11   3   1
B3              3   1   1   2   1   0
S1             19  10   2  20  10   2
S2             11   4   0   7   4   0
S3              2

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [15]:
porcentaje_observado = [None] * numero_periodos

for i in range(0, numero_periodos):
    # Calcula los porcentajes observados dividiendo las frecuencias observadas por la suma total de cada periodo
    porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
    #Transpone la matriz para que sea más fácil de leer y comparar con el Excel
    porcentaje_observado[i] = porcentaje_observado[i].T
    
print("Porcentaje relativo:\n", porcentaje_observado[2]) #Ejemplo de impresion de un periodo


Porcentaje relativo:
 Current State        B1        B2        B3        S1        S2        S3
Current State                                                            
B1             0.373134  0.104478  0.044776  0.283582  0.164179  0.029851
B2             0.360000  0.040000  0.040000  0.400000  0.160000  0.000000
B3             0.375000  0.250000  0.125000  0.250000  0.000000  0.000000
S1             0.349206  0.174603  0.031746  0.317460  0.111111  0.015873
S2             0.307692  0.115385  0.038462  0.384615  0.153846  0.000000
S3             0.000000  0.333333  0.000000  0.666667  0.000000  0.000000


In [16]:
frecuencias_observadas = []
porcentaje_observado = [None] * numero_periodos

#Se calcula la tabla de frecuencias observadas para cada periodo
for i in range(numero_periodos):
    frecuencia_obs = pd.crosstab(table_by_period[i]['Current State'].shift(), table_by_period[i]['Current State'], normalize=False)
    frecuencias_obs = np.array(frecuencia_obs)    
    frecuencias_observadas.append(frecuencia_obs)


def calculateChiArgs(data):
    frecuencias_observadasPeriodo = []

    #PROBABILIDAD ESPERADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #SE CREA LA MATRIZ DE FRECUENCIAS ESPERADAS
    freq_expected=np.zeros(((len(states_mapping)),len(states_mapping)))
    for i in range(len(data)):
        freq_expected[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    
    freq_obs = np.zeros(((len(states_mapping)),len(states_mapping)))
    index_to_delete = []
    #FUNCION PARA FRECUENCIAS DE 0
    if 0 in freq_expected:
        freq_expected, states, index_to_delete =resizeExpectedValues(freq_expected,states)
        states_mapping={new_state: i for i,new_state in enumerate(states)}
    
    rows = ['B1', 'B2', 'B3', 'S1', 'S2', 'S3']
    
    for i in range(0, numero_periodos):
        #Calculo de Frecuencias Observadas por periodo
        frecuencias_observadas[i] = resizeExpectedValuesObs(frecuencias_observadas[i].values, index_to_delete)
        #Calculo de porcentaje observado
        porcentaje_observado[i] = frecuencias_observadas[i] / frecuencias_observadas[i].sum()
        porcentaje_observado[i] = pd.DataFrame(porcentaje_observado[i], columns=rows, index=rows)
        print(f"Porcentaje observado:\n{porcentaje_observado[i]}\n") #Ejemplo de impresion de un periodo
        porcentaje_observado[i] = porcentaje_observado[i].values

    #SE DETERMINA LA MATRIZ DE PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states)), len(states)))
    for i in range(len(freq_expected)):
        transition_expected[i] = freq_expected[i] / sum(freq_expected[i])

    data_frame_expected = pd.DataFrame(transition_expected, columns=states, index=states)
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")
    return transition_expected
    

#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):

    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if i > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states, index_to_delete

#Cambia el tamaño de la matriz de frecuencias observadas para mantener disposición de columnas y filas reducidas en frecuencias esperadas
def resizeExpectedValuesObs(freq_matrix, index_to_delete):
    for i in range(len(freq_matrix)):
        if i in index_to_delete:
            if i > 0:
                
                freq_matrix[i-1]=freq_matrix[i-1]+freq_matrix[i]
                freq_matrix[:,i-1] = freq_matrix[:,i-1] + freq_matrix[:,i]
            else:
                freq_matrix[i+1]=freq_matrix[i]+freq_matrix[i+1]
                freq_matrix[:,i+1] = freq_matrix[:,i+1] + freq_matrix[:,i]
        
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    return freq_matrix
probabilidad_observada = []
probabilidad_observada = calculateChiArgs(ford_trim[['Previous State', 'Current State', 'Next State']])

Porcentaje observado:
          B1        B2        B3        S1        S2        S3
B1  0.135417  0.000000  0.020833  0.145833  0.036458  0.005208
B2  0.031250  0.026042  0.005208  0.010417  0.005208  0.010417
B3  0.010417  0.000000  0.000000  0.015625  0.015625  0.000000
S1  0.135417  0.036458  0.010417  0.130208  0.041667  0.000000
S2  0.026042  0.015625  0.005208  0.036458  0.031250  0.020833
S3  0.005208  0.010417  0.000000  0.015625  0.005208  0.000000

Porcentaje observado:
          B1        B2        B3        S1        S2        S3
B1  0.057292  0.041667  0.000000  0.161458  0.026042  0.000000
B2  0.026042  0.005208  0.005208  0.046875  0.026042  0.005208
B3  0.005208  0.010417  0.000000  0.010417  0.000000  0.000000
S1  0.140625  0.036458  0.020833  0.119792  0.062500  0.015625
S2  0.046875  0.015625  0.005208  0.052083  0.020833  0.005208
S3  0.010417  0.005208  0.000000  0.005208  0.005208  0.005208

Porcentaje observado:
          B1        B2        B3        S1        

In [17]:
ChiObs = 0

for i in range(0, numero_periodos): 
     # Suma acumulativa de los valores chi-cuadrado observados se aplica la formula vista en clase
    ChiObs+= np.sum(frecuencias_observadas[i].sum()*(porcentaje_observado[i]-probabilidad_observada)**2/probabilidad_observada)
    break
print(f'Chi Observado: {ChiObs}')


Chi Observado: 826.4377703963524


In [18]:
from scipy.stats import chi2
grados_libertad_total = 0
sum_filas = 0
# Iteración a través de las frecuencias observadas
for df in frecuencias_observadas:
    num_filas, num_columnas = df.shape
    # Suma el número de filas al total para los grados de libertad
    sum_filas += num_filas
# Calcula los grados de libertad
grados_libertad = (sum_filas - 1) * (num_columnas - 1)

print("Grados de libertad total:", grados_libertad)
# Calcula el valor chi-cuadrado teórico para un nivel de significancia del 5% y los grados de libertad calculados
chiTeorico = chi2.ppf(0.95, grados_libertad)
print("Chi Teorico= ",chiTeorico)

Grados de libertad total: 145
Chi Teorico=  174.10098056972615


In [19]:
# Compara el valor chi-cuadrado observado con el valor chi-cuadrado teórico crítico
if(ChiObs < chiTeorico):
    print("Se acepta la hipótesis nula, por lo tanto se considera estacionario en el tiempo.")
else:
    print("Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.")

Se rechaza la hipótesis nula, por lo tanto no se considera estacionario en el tiempo.
